## Import Library

In [ ]:
# These are required
!git clone https://github.com/polimi-ispl/icpr2020dfdc
!pip install efficientnet-pytorch
!pip install -U git+https://github.com/albu/albumentations > /dev/null

In [ ]:
import torch
from torch.utils.model_zoo import load_url

from PIL import Image
import matplotlib.pyplot as plt
import os
import sys
sys.path.append('..')

%cd icpr2020dfdc/notebook
from blazeface import FaceExtractor, BlazeFace
from architectures import fornet,weights
from isplutils import utils
%cd ..
%cd ..

from tqdm.notebook import tqdm_notebook as tqdm
import numpy as np
import pandas as pd

In [ ]:
# Read in data
real = "../input/real-and-fake-face/training/real"
fake = "../input/real-and-fake-face/training/fake"

real_path = os.listdir(real)
fake_path = os.listdir(fake)

## Load efficientNet

In [ ]:
device = torch.device('cuda:0') if torch.cuda.is_available() else torch.device('cpu')
face_policy = 'scale'
face_size = 224

# net_model = 'EfficientNetB4'
net_model = 'EfficientNetAutoAttB4ST'

train_db = 'DFDC'
model_url = weights.weight_url['{:s}_{:s}'.format(net_model,train_db)]
net = getattr(fornet,net_model)().eval().to(device)
net.load_state_dict(load_url(model_url,map_location=device,check_hash=True))
transf = utils.get_transformer(face_policy, face_size, net.get_normalizer(), train=False)

%cd icpr2020dfdc/notebook
facedet = BlazeFace().to(device)
facedet.load_weights("../blazeface/blazeface.pth")
facedet.load_anchors("../blazeface/anchors.npy")
face_extractor = FaceExtractor(facedet=facedet)
%cd ..
%cd ..

## Load Real-Fake images and do the prediction

In [ ]:
im_num = len(fake_path)
im_num

In [ ]:
# Threshold is set to be 0.2
TP = 0
TN = 0
FN = 0
FP = 0

for i in tqdm(range(im_num)):
    
    im_fake = Image.open(fake + '/' + fake_path[i])
    im_real = Image.open(real + '/' + real_path[i])

    im_real_faces = face_extractor.process_image(img=im_real)
    im_fake_faces = face_extractor.process_image(img=im_fake)
    
    try:
        # take the face with the highest confidence score found by BlazeFace
        im_real_face = im_real_faces['faces'][0] 
        im_fake_face = im_fake_faces['faces'][0]
        
    except IndexError: # Sometimes this will be empty
        continue

    faces_t = torch.stack([transf(image=im)['image'] for im in [im_real_face,im_fake_face]])

    with torch.no_grad():
        faces_pred = torch.sigmoid(net(faces_t.to(device))).cpu().numpy().flatten()
        
    if faces_pred[0] <= 0.2:
        TP += 1
    else:
        FP += 1

    if faces_pred[1] <= 0.2:
        FN += 1
    else:
        TN += 1
        
    if (i + 1) % 100 == 0:
        
        acc = (TP + TN) / (TP + TN + FN + FP)
        precision = TP / (TP + FP)
        recall = TP / (TP + FN)
        f1 = (precision * recall * 2) / (precision + recall)
        
        print('|Accuracy: {:.4f} | Precision: {:.4f} | Recall: {:.4f}| F1-Score: {:.4f} |'.format(acc, precision, recall, f1))
        print('==========================================================================')